## XGB to SQL

Rather than running predictions from a saved Xgboost model, it's now possible to run model predictions in the database.

Let's try this out by loading the kddcup99 dataset into a SQLITE database.

Here's a description of the dataset.

In [1]:
from sklearn.datasets import fetch_kddcup99

data = fetch_kddcup99(subset = 'SA')

print(data['DESCR'])

.. _kddcup99_dataset:

Kddcup 99 dataset
-----------------

The KDD Cup '99 dataset was created by processing the tcpdump portions
of the 1998 DARPA Intrusion Detection System (IDS) Evaluation dataset,
created by MIT Lincoln Lab [1]. The artificial data (described on the `dataset's
homepage <https://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html>`_) was
generated using a closed network and hand-injected attacks to produce a
large number of different types of attack with normal activity in the
background. As the initial goal was to produce a large training set for
supervised learning algorithms, there is a large proportion (80.1%) of
abnormal data which is unrealistic in real world, and inappropriate for
unsupervised anomaly detection which aims at detecting 'abnormal' data, ie

1) qualitatively different from normal data

2) in large minority among the observations.

We thus transform the KDD Data set into two different data sets: SA and SF.

-SA is obtained by simply selecting all t

Here, I'm using the SA structure of the dataset.

Now, the data isn't exactly ready to train a machine learning model on.

The data has to be cleaned up first, e.g. the text must be converted to numbers.

Let's do that.

In [2]:
import pandas as pd

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999


df = pd.DataFrame(data["data"])
df["y"] = data["target"]
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,y
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0,0,0,0,1,0,0,9,9,1,0,0.11,0,0,0,0,0,b'normal.'
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0,0,0,0,1,0,0,19,19,1,0,0.05,0,0,0,0,0,b'normal.'
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0,0,0,0,1,0,0,29,29,1,0,0.03,0,0,0,0,0,b'normal.'
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0,0,0,0,1,0,0,39,39,1,0,0.03,0,0,0,0,0,b'normal.'
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0,0,0,0,1,0,0,49,49,1,0,0.02,0,0,0,0,0,b'normal.'


Check for missing values.

In [3]:
df.isnull().any().sum()

0

Rename columns.

In [4]:
cols = df.columns.tolist()
cols = ['x' + str(col) for col in cols]
cols[-1] = 'y'

df.columns = cols
print(df.columns)

Index(['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10',
       'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20',
       'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30',
       'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39', 'x40',
       'y'],
      dtype='object')


Label encode the columns with text.

In [5]:
def fix_x1(x):
    if x == b'tcp':
        return 0
    elif x == b'udp':
        return 1
    else:
        return 2
    
df["x1"] = df["x1"].apply(fix_x1)

In [6]:
def fix_x2(x):
    if x == b'http':
        return 0
    elif x == b'smtp':
        return 1
    elif x == b'private':
        return 2
    elif x == b'domain_u':
        return 3
    elif x == b'other':
        return 4
    elif x == b'ftp_data':
        return 5
    elif x ==  b'ecr_i':
        return 6
    elif x == b'urp_i':
        return 7
    elif x == b'finger':
        return 8
    elif x == b'eco_i':
        return 9
    elif x == b'ntp_u':
        return 10
    elif x == b'ftp':
        return 11
    elif x == b'auth':
        return 12
    elif x == b'telnet':
        return 13
    else:
        return 14
    
df["x2"] = df["x2"].apply(fix_x2)

In [7]:
def fix_x3(x):
    if x == b'SF':
        return 0
    elif x == b'REJ':
        return 1
    elif x == b'S0':
        return 2
    else:
        return 3
    
df["x3"] = df["x3"].apply(fix_x3)

Fix the dependent variable.

In [8]:
df["y"].value_counts()

b'normal.'         97278
b'smurf.'           2338
b'neptune.'          973
b'back.'              16
b'portsweep.'         13
b'satan.'             11
b'ipsweep.'            9
b'teardrop.'           7
b'warezclient.'        5
b'nmap.'               3
b'pod.'                1
b'imap.'               1
Name: y, dtype: int64

In [9]:
def fix_y(x):
    if x == b'normal.':
        return 0
    else:
        return 1

df["y"] = df["y"].apply(fix_y)

Check the column types.

In [10]:
df.dtypes

x0     object
x1      int64
x2      int64
x3      int64
x4     object
x5     object
x6     object
x7     object
x8     object
x9     object
x10    object
x11    object
x12    object
x13    object
x14    object
x15    object
x16    object
x17    object
x18    object
x19    object
x20    object
x21    object
x22    object
x23    object
x24    object
x25    object
x26    object
x27    object
x28    object
x29    object
x30    object
x31    object
x32    object
x33    object
x34    object
x35    object
x36    object
x37    object
x38    object
x39    object
x40    object
y       int64
dtype: object

Change datatypes to integer or float.

In [11]:
df["x0"] = df["x0"].astype(int)
df["x4"] = df["x4"].astype(int)
df["x5"] = df["x5"].astype(int)
df["x6"] = df["x6"].astype(int)
df["x7"] = df["x7"].astype(int)
df["x8"] = df["x8"].astype(int)
df["x9"] = df["x9"].astype(int)
df["x10"] = df["x10"].astype(int)
df["x11"] = df["x11"].astype(int)
df["x12"] = df["x12"].astype(int)
df["x13"] = df["x13"].astype(int)
df["x14"] = df["x14"].astype(int)
df["x15"] = df["x15"].astype(int)
df["x16"] = df["x16"].astype(int)
df["x17"] = df["x17"].astype(int)
df["x18"] = df["x18"].astype(int)
df["x19"] = df["x19"].astype(int)
df["x20"] = df["x20"].astype(int)
df["x21"] = df["x21"].astype(int)
df["x22"] = df["x22"].astype(int)
df["x23"] = df["x23"].astype(int)
df["x24"] = df["x24"].astype(float)
df["x25"] = df["x25"].astype(float)
df["x26"] = df["x26"].astype(float)
df["x27"] = df["x27"].astype(float)
df["x28"] = df["x28"].astype(float)
df["x29"] = df["x29"].astype(float)
df["x30"] = df["x30"].astype(float)
df["x31"] = df["x31"].astype(int)
df["x32"] = df["x32"].astype(int)
df["x33"] = df["x33"].astype(float)
df["x34"] = df["x34"].astype(float)
df["x35"] = df["x35"].astype(float)
df["x36"] = df["x36"].astype(float)
df["x37"] = df["x37"].astype(float)
df["x38"] = df["x38"].astype(float)
df["x39"] = df["x39"].astype(float)
df["x40"] = df["x40"].astype(float)

Create the database if it doesn't already exist.

In [12]:
import sqlite3

conn = sqlite3.connect("kddcup99_db")
print(sqlite3.version)

2.6.0


Drop the table, if it exists.

In [13]:
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS clean_kdd;")


Create the table.

In [14]:
command = """
CREATE TABLE clean_kdd (
    x0 INTEGER,
    x1 INTEGER,
    x2 INTEGER,
    x3 INTEGER,
    x4 INTEGER,
    x5 INTEGER,
    x6 INTEGER,
    x7 INTEGER,
    x8 INTEGER,
    x9 INTEGER,
    x10 INTEGER,
    x11 INTEGER,
    x12 INTEGER,
    x13 INTEGER,
    x14 INTEGER,
    x15 INTEGER,
    x16 INTEGER,
    x17 INTEGER,
    x18 INTEGER,
    x19 INTEGER,
    x20 INTEGER,
    x21 INTEGER,
    x22 INTEGER,
    x23 INTEGER,
    x24 REAL,
    x25 REAL,
    x26 REAL,
    x27 REAL,
    x28 REAL,
    x29 REAL,
    x30 REAL,
    x31 INTEGER,
    x32 INTEGER,
    x33 REAL,
    x34 REAL,
    x35 REAL,
    x36 REAL,
    x37 REAL,
    x38 REAL,
    x39 REAL,
    x40 REAL,
    y INTEGER
    );
"""

cursor.execute(command)
conn.commit()

Insert values.

In [15]:
command = """
INSERT INTO clean_kdd VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
                           ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""

cursor.executemany(command, df.values.tolist())

conn.commit()

As a check, query the 1st row.

In [16]:
query = "SELECT * FROM clean_kdd LIMIT 1;"

cursor.execute(query)
cursor.fetchall()

[(0,
  0,
  0,
  0,
  181,
  5450,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  8,
  8,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  9,
  9,
  1.0,
  0.0,
  0.11,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0)]

Train an Xgboost model to the dataset.

In [17]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(df.iloc[:, 0:41], df["y"])

XGBClassifier()

Using the xgb2sql function to turn model into a SQL query.

Let's see the query.

In [18]:
from xgb2sql import xgb2sql

query = xgb2sql(model.get_booster(), 'clean_kdd')
print(query)

WITH booster_output AS (
	SELECT
		CASE
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x7 < 1) OR (x7 IS NULL))
		THEN -0.199764431
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x7 >= 1)
		THEN 0.127272725
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x3 < 3) OR (x3 IS NULL))
		THEN 0.174193546
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x3 >= 3)
		THEN -0.100000001
			WHEN (x22 >= 101)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.199393019
			WHEN (x22 >= 101)
			AND (x5 >= 2)
		THEN -0.198542818
		END AS column_0, 
		CASE
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x7 < 1) OR (x7 IS NULL))
		THEN -0.181655183
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x7 >= 1)
		THEN 0.119491577
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.9

A few things to notice here...

There are 100 columns, for each of the 100 trees in the Xgboost model.

Also, this query uses the EXP function, which is not available in SQLITE. 

Thus, to run model predictions, I'll query output from the 100 trees, then compute the sigmoid function on the sum of each row.

In [19]:
query = """
SELECT
		CASE
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x7 < 1) OR (x7 IS NULL))
		THEN -0.199764431
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x7 >= 1)
		THEN 0.127272725
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x3 < 3) OR (x3 IS NULL))
		THEN 0.174193546
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x3 >= 3)
		THEN -0.100000001
			WHEN (x22 >= 101)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.199393019
			WHEN (x22 >= 101)
			AND (x5 >= 2)
		THEN -0.198542818
		END AS column_0, 
		CASE
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x7 < 1) OR (x7 IS NULL))
		THEN -0.181655183
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x7 >= 1)
		THEN 0.119491577
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x35 < 0.00499999989) OR (x35 IS NULL))
		THEN 0.159316763
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x35 >= 0.00499999989)
		THEN -0.0642817393
			WHEN (x22 >= 101)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.181331933
			WHEN (x22 >= 101)
			AND (x5 >= 2)
		THEN -0.180653468
		END AS column_1, 
		CASE
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x7 < 1) OR (x7 IS NULL))
		THEN -0.168046236
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x7 >= 1)
		THEN 0.112729825
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x35 < 0.00499999989) OR (x35 IS NULL))
		THEN 0.148200467
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x35 >= 0.00499999989)
		THEN -0.0608035214
			WHEN (x22 >= 101)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.1677472
			WHEN (x22 >= 101)
			AND (x5 >= 2)
		THEN -0.167169586
		END AS column_2, 
		CASE
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x7 < 1) OR (x7 IS NULL))
		THEN -0.157473892
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x7 >= 1)
		THEN 0.106788218
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x35 < 0.00499999989) OR (x35 IS NULL))
		THEN 0.139105961
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x35 >= 0.00499999989)
		THEN -0.057575684
			WHEN (x22 >= 101)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.15718469
			WHEN (x22 >= 101)
			AND (x5 >= 2)
		THEN -0.156667218
		END AS column_3, 
		CASE
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x7 < 1) OR (x7 IS NULL))
		THEN -0.149051234
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x7 >= 1)
		THEN 0.10151428
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x35 < 0.00499999989) OR (x35 IS NULL))
		THEN 0.131506026
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x35 >= 0.00499999989)
		THEN -0.05457104
			WHEN (x22 >= 101)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.148762211
			WHEN (x22 >= 101)
			AND (x5 >= 2)
		THEN -0.148279756
		END AS column_4, 
		CASE
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x7 < 1) OR (x7 IS NULL))
		THEN -0.142207608
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x7 >= 1)
		THEN 0.0967899337
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x35 < 0.00499999989) OR (x35 IS NULL))
		THEN 0.12503773
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x35 >= 0.00499999989)
		THEN -0.0517664663
			WHEN (x22 >= 101)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.141911775
			WHEN (x22 >= 101)
			AND (x5 >= 2)
		THEN -0.141447529
		END AS column_5, 
		CASE
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x12 < 1) OR (x12 IS NULL))
		THEN -0.136613116
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x12 >= 1)
		THEN -0.041330643
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x32 < 2) OR (x32 IS NULL))
		THEN -0.0432456434
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x32 >= 2)
		THEN 0.121677555
			WHEN (x22 >= 101)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.136250094
			WHEN (x22 >= 101)
			AND (x5 >= 2)
		THEN -0.135792181
		END AS column_6, 
		CASE
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x12 < 1) OR (x12 IS NULL))
		THEN -0.131889462
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x12 >= 1)
		THEN -0.0381667353
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x35 < 0.00499999989) OR (x35 IS NULL))
		THEN 0.11500404
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x35 >= 0.00499999989)
		THEN -0.0488083698
			WHEN (x22 >= 101)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.131508633
			WHEN (x22 >= 101)
			AND (x5 >= 2)
		THEN -0.131048128
		END AS column_7, 
		CASE
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x7 < 1) OR (x7 IS NULL))
		THEN -0.127842411
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x7 >= 1)
		THEN 0.104944088
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x32 < 2) OR (x32 IS NULL))
		THEN -0.0413303226
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x32 >= 2)
		THEN 0.112706833
			WHEN (x22 >= 101)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.127493322
			WHEN (x22 >= 101)
			AND (x5 >= 2)
		THEN -0.127023116
		END AS column_8, 
		CASE
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x12 < 1) OR (x12 IS NULL))
		THEN -0.124496244
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x12 >= 1)
		THEN -0.0250920653
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x35 < 0.00499999989) OR (x35 IS NULL))
		THEN 0.107106864
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x35 >= 0.00499999989)
		THEN -0.0460923724
			WHEN (x22 >= 101)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.124060228
			WHEN (x22 >= 101)
			AND (x5 >= 2)
		THEN -0.123574339
		END AS column_9, 
		CASE
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x12 < 1) OR (x12 IS NULL))
		THEN -0.121568523
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x12 >= 1)
		THEN -0.0230523199
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x32 < 2) OR (x32 IS NULL))
		THEN -0.0395141393
			WHEN ((x22 < 101) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x32 >= 2)
		THEN 0.1055158
			WHEN (x22 >= 101)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.121100165
			WHEN (x22 >= 101)
			AND (x5 >= 2)
		THEN -0.120593369
		END AS column_10, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
		THEN -0.119057335
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x4 >= 53417)
		THEN -0.0324516706
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x35 < 0.00499999989) OR (x35 IS NULL))
		THEN 0.100613587
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x35 >= 0.00499999989)
		THEN -0.0435985699
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.118679047
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.118010998
		END AS column_11, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x12 < 1) OR (x12 IS NULL))
		THEN -0.11683692
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x12 >= 1)
		THEN -0.0124968532
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x32 < 2) OR (x32 IS NULL))
		THEN -0.0378041156
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x32 >= 2)
		THEN 0.0995104685
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.116433434
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.115732387
		END AS column_12, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x34 < 0.995000005) OR (x34 IS NULL))
		THEN -0.114863053
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x34 >= 0.995000005)
		THEN 0.0489219353
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x35 < 0.00499999989) OR (x35 IS NULL))
		THEN 0.0950736627
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x35 >= 0.00499999989)
		THEN -0.041306328
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.114457488
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.11371851
		END AS column_13, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x12 < 1) OR (x12 IS NULL))
			AND ((x7 < 1) OR (x7 IS NULL))
		THEN -0.113178849
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x12 < 1) OR (x12 IS NULL))
			AND (x7 >= 1)
		THEN 0.13355875
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x12 >= 1)
			AND ((x4 < 30602) OR (x4 IS NULL))
		THEN -0.111183718
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x12 >= 1)
			AND (x4 >= 30602)
		THEN 0.358976692
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.112708032
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.111926414
		END AS column_14, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x7 < 1) OR (x7 IS NULL))
		THEN -0.111756019
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x7 >= 1)
		THEN 0.124898136
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x34 < 0.504999995) OR (x34 IS NULL))
		THEN -0.0650386214
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x34 >= 0.504999995)
		THEN 0.402898163
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.111149631
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.110320769
		END AS column_15, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
		THEN -0.110395811
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x38 < 0.985000014) OR (x38 IS NULL))
			AND (x4 >= 53417)
		THEN -0.000105130486
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND ((x32 < 2) OR (x32 IS NULL))
		THEN -0.0267156065
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x38 >= 0.985000014)
			AND (x32 >= 2)
		THEN 0.0997123495
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.109752886
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.108872212
		END AS column_16, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x12 < 1) OR (x12 IS NULL))
		THEN -0.109338097
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x12 >= 1)
		THEN 0.018069556
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x33 < 0.00499999989) OR (x33 IS NULL))
		THEN 0.318744391
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x33 >= 0.00499999989)
		THEN -0.0617661364
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.10849309
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.107556
		END AS column_17, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x12 < 1) OR (x12 IS NULL))
		THEN -0.108283758
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x12 >= 1)
		THEN 0.0162363015
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x31 < 254) OR (x31 IS NULL))
		THEN -0.0990222469
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x31 >= 254)
		THEN 0.125791043
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.107349269
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.106351137
		END AS column_18, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND ((x34 < 0.995000005) OR (x34 IS NULL))
		THEN -0.107194319
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND (x34 >= 0.995000005)
		THEN 0.0803208351
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND ((x2 < 1) OR (x2 IS NULL))
		THEN 0.31506905
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND (x2 >= 1)
		THEN -0.0988944247
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.106303446
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.105239585
		END AS column_19, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x7 < 1) OR (x7 IS NULL))
		THEN -0.106379151
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x7 >= 1)
		THEN 0.146216974
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x31 < 254) OR (x31 IS NULL))
		THEN -0.0950858817
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x31 >= 254)
		THEN 0.117601492
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.105340041
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.104205661
		END AS column_20, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND ((x34 < 0.995000005) OR (x34 IS NULL))
		THEN -0.105550148
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND (x34 >= 0.995000005)
		THEN 0.0758197308
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND ((x2 < 1) OR (x2 IS NULL))
		THEN 0.272754163
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND (x2 >= 1)
		THEN -0.0955388546
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x34 < 0.0850000009) OR (x34 IS NULL))
		THEN 0.104958549
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x34 >= 0.0850000009)
		THEN 0.0491344295
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.103235662
		END AS column_21, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x12 < 1) OR (x12 IS NULL))
		THEN -0.105026595
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x12 >= 1)
		THEN 0.023924144
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x33 < 0.075000003) OR (x33 IS NULL))
		THEN 0.142961875
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x33 >= 0.075000003)
		THEN -0.0765725002
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x28 < 0.0149999997) OR (x28 IS NULL))
		THEN 0.0644141659
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x28 >= 0.0149999997)
		THEN 0.104738474
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.10231749
		END AS column_22, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x7 < 1) OR (x7 IS NULL))
		THEN -0.104250908
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x7 >= 1)
		THEN 0.151191235
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x31 < 254) OR (x31 IS NULL))
		THEN -0.090278767
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x31 >= 254)
		THEN 0.108813994
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x28 < 0.0149999997) OR (x28 IS NULL))
		THEN 0.0619995184
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x28 >= 0.0149999997)
		THEN 0.104047909
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.101440348
		END AS column_23, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND ((x36 < 0.49000001) OR (x36 IS NULL))
		THEN -0.103636108
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND (x36 >= 0.49000001)
		THEN 0.0636340901
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND ((x2 < 1) OR (x2 IS NULL))
		THEN 0.24106133
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND (x2 >= 1)
		THEN -0.09059003
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x28 < 0.0149999997) OR (x28 IS NULL))
		THEN 0.0596000627
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x28 >= 0.0149999997)
		THEN 0.103404656
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.100594521
		END AS column_24, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
		THEN -0.103275456
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x4 >= 53417)
		THEN 0.0309525318
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x33 < 0.075000003) OR (x33 IS NULL))
		THEN 0.136818498
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x33 >= 0.075000003)
		THEN -0.0700135902
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x34 < 0.0850000009) OR (x34 IS NULL))
		THEN 0.102075651
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x34 >= 0.0850000009)
		THEN 0.0355837755
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.0997712389
		END AS column_25, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x36 < 0.49000001) OR (x36 IS NULL))
		THEN -0.102706574
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x36 >= 0.49000001)
		THEN 0.0705130324
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x31 < 254) OR (x31 IS NULL))
		THEN -0.0855142251
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x31 >= 254)
		THEN 0.104478262
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x28 < 0.0149999997) OR (x28 IS NULL))
		THEN 0.0533680283
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x28 >= 0.0149999997)
		THEN 0.102236807
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.0989625305
		END AS column_26, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND ((x7 < 1) OR (x7 IS NULL))
		THEN -0.101970091
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND (x7 >= 1)
		THEN 0.160398304
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND ((x2 < 1) OR (x2 IS NULL))
		THEN 0.216884002
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND (x2 >= 1)
		THEN -0.0878415033
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x28 < 0.0149999997) OR (x28 IS NULL))
		THEN 0.0510644279
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x28 >= 0.0149999997)
		THEN 0.101690091
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.098161079
		END AS column_27, 
		CASE
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x36 < 0.49000001) OR (x36 IS NULL))
		THEN -0.101798169
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x36 >= 0.49000001)
		THEN 0.0736828223
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x33 < 0.075000003) OR (x33 IS NULL))
		THEN 0.131243989
			WHEN ((x22 < 100) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x33 >= 0.075000003)
		THEN -0.0635812506
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x28 < 0.0149999997) OR (x28 IS NULL))
		THEN 0.048799105
			WHEN (x22 >= 100)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x28 >= 0.0149999997)
		THEN 0.101162948
			WHEN (x22 >= 100)
			AND (x5 >= 2)
		THEN -0.097360149
		END AS column_28, 
		CASE
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND ((x34 < 0.995000005) OR (x34 IS NULL))
		THEN -0.101492442
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND (x34 >= 0.995000005)
		THEN 0.102769174
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND ((x2 < 1) OR (x2 IS NULL))
		THEN 0.202735767
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND (x2 >= 1)
		THEN -0.0845451877
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x22 < 84) OR (x22 IS NULL))
		THEN 0.305251688
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x22 >= 84)
		THEN 0.0986718759
			WHEN (x22 >= 67)
			AND (x5 >= 2)
		THEN -0.0987924859
		END AS column_29, 
		CASE
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND ((x36 < 0.49000001) OR (x36 IS NULL))
		THEN -0.101151086
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND (x36 >= 0.49000001)
		THEN 0.0737398639
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND ((x2 < 1) OR (x2 IS NULL))
		THEN 0.181552559
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND (x2 >= 1)
		THEN -0.0830125585
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x22 < 84) OR (x22 IS NULL))
		THEN 0.298047155
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x22 >= 84)
		THEN 0.0979672968
			WHEN (x22 >= 67)
			AND (x5 >= 2)
		THEN -0.0981531069
		END AS column_30, 
		CASE
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x12 < 1) OR (x12 IS NULL))
		THEN -0.100962952
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x12 >= 1)
		THEN 0.0507909022
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x33 < 0.00499999989) OR (x33 IS NULL))
		THEN 0.198292226
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x33 >= 0.00499999989)
		THEN -0.0369720459
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x22 < 84) OR (x22 IS NULL))
		THEN 0.289634496
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x22 >= 84)
		THEN 0.097249791
			WHEN (x22 >= 67)
			AND (x5 >= 2)
		THEN -0.0975079909
		END AS column_31, 
		CASE
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND ((x36 < 0.49000001) OR (x36 IS NULL))
		THEN -0.100390173
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND (x36 >= 0.49000001)
		THEN 0.0766674802
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND ((x2 < 1) OR (x2 IS NULL))
		THEN 0.161935747
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND (x2 >= 1)
		THEN -0.079559736
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x22 < 84) OR (x22 IS NULL))
		THEN 0.279845327
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x22 >= 84)
		THEN 0.0965135396
			WHEN (x22 >= 67)
			AND (x5 >= 2)
		THEN -0.0968521759
		END AS column_32, 
		CASE
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x36 < 0.49000001) OR (x36 IS NULL))
		THEN -0.100250021
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x36 >= 0.49000001)
		THEN 0.0681897476
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x33 < 0.00499999989) OR (x33 IS NULL))
		THEN 0.18841204
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x33 >= 0.00499999989)
		THEN -0.0282180589
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x22 < 84) OR (x22 IS NULL))
		THEN 0.268392116
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x22 >= 84)
		THEN 0.0957528576
			WHEN (x22 >= 67)
			AND (x5 >= 2)
		THEN -0.0961809829
		END AS column_33, 
		CASE
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
		THEN -0.0999359116
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x4 >= 53417)
		THEN 0.0542854369
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x31 < 254) OR (x31 IS NULL))
		THEN -0.0712999031
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x31 >= 254)
		THEN 0.107317984
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x22 < 84) OR (x22 IS NULL))
		THEN 0.255104154
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x22 >= 84)
		THEN 0.0949622616
			WHEN (x22 >= 67)
			AND (x5 >= 2)
		THEN -0.0954899788
		END AS column_34, 
		CASE
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x36 < 0.49000001) OR (x36 IS NULL))
		THEN -0.0995551497
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x36 >= 0.49000001)
		THEN 0.0724600405
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x35 < 0.689999998) OR (x35 IS NULL))
		THEN -0.0310507547
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x35 >= 0.689999998)
		THEN 0.160233781
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x22 < 84) OR (x22 IS NULL))
		THEN 0.240181133
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x22 >= 84)
		THEN 0.0941364318
			WHEN (x22 >= 67)
			AND (x5 >= 2)
		THEN -0.0947750285
		END AS column_35, 
		CASE
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND ((x36 < 0.49000001) OR (x36 IS NULL))
		THEN -0.0989708006
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND (x36 >= 0.49000001)
		THEN 0.0612483732
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND ((x2 < 1) OR (x2 IS NULL))
		THEN 0.158894584
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND (x2 >= 1)
		THEN -0.0750202313
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x22 < 84) OR (x22 IS NULL))
		THEN 0.224233016
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x22 >= 84)
		THEN 0.0932701305
			WHEN (x22 >= 67)
			AND (x5 >= 2)
		THEN -0.0940322503
		END AS column_36, 
		CASE
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x9 < 2) OR (x9 IS NULL))
		THEN -0.0991668925
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x9 >= 2)
		THEN 0.040474508
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x32 < 17) OR (x32 IS NULL))
		THEN 0.112894297
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x32 >= 17)
		THEN -0.0530888513
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x22 < 84) OR (x22 IS NULL))
		THEN 0.208078578
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x22 >= 84)
		THEN 0.092358321
			WHEN (x22 >= 67)
			AND (x5 >= 2)
		THEN -0.0932580456
		END AS column_37, 
		CASE
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x9 < 2) OR (x9 IS NULL))
		THEN -0.0988293216
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x9 >= 2)
		THEN 0.0360590816
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x31 < 254) OR (x31 IS NULL))
		THEN -0.0660855994
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x31 >= 254)
		THEN 0.0984944925
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x22 < 79) OR (x22 IS NULL))
		THEN 0.198894292
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x22 >= 79)
		THEN 0.0914709195
			WHEN (x22 >= 67)
			AND (x5 >= 2)
		THEN -0.0924491584
		END AS column_38, 
		CASE
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND ((x36 < 0.49000001) OR (x36 IS NULL))
		THEN -0.097873129
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND (x36 >= 0.49000001)
		THEN 0.0784746855
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND ((x2 < 1) OR (x2 IS NULL))
		THEN 0.142242238
			WHEN ((x22 < 67) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND (x2 >= 1)
		THEN -0.0690978542
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x28 < 0.0350000001) OR (x28 IS NULL))
		THEN 0.0321669839
			WHEN (x22 >= 67)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x28 >= 0.0350000001)
		THEN 0.104517713
			WHEN (x22 >= 67)
			AND (x5 >= 2)
		THEN -0.0916026309
		END AS column_39, 
		CASE
			WHEN ((x22 < 62) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x36 < 0.49000001) OR (x36 IS NULL))
		THEN -0.0978600681
			WHEN ((x22 < 62) OR (x22 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x36 >= 0.49000001)
		THEN 0.0698130056
			WHEN ((x22 < 62) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND ((x35 < 0.689999998) OR (x35 IS NULL))
		THEN -0.0308703519
			WHEN ((x22 < 62) OR (x22 IS NULL))
			AND (x3 >= 2)
			AND (x35 >= 0.689999998)
		THEN 0.151207149
			WHEN (x22 >= 62)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x28 < 0.0350000001) OR (x28 IS NULL))
		THEN 0.0303133857
			WHEN (x22 >= 62)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x28 >= 0.0350000001)
		THEN 0.105708972
			WHEN (x22 >= 62)
			AND (x5 >= 2)
		THEN -0.0916181505
		END AS column_40, 
		CASE
			WHEN ((x22 < 62) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND ((x34 < 0.995000005) OR (x34 IS NULL))
		THEN -0.0968134552
			WHEN ((x22 < 62) OR (x22 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND (x34 >= 0.995000005)
		THEN 0.110531189
			WHEN ((x22 < 62) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND ((x2 < 1) OR (x2 IS NULL))
		THEN 0.138930544
			WHEN ((x22 < 62) OR (x22 IS NULL))
			AND (x4 >= 53417)
			AND (x2 >= 1)
		THEN -0.0652647763
			WHEN (x22 >= 62)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x28 < 0.0350000001) OR (x28 IS NULL))
		THEN 0.0285370648
			WHEN (x22 >= 62)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x28 >= 0.0350000001)
		THEN 0.105515778
			WHEN (x22 >= 62)
			AND (x5 >= 2)
		THEN -0.0907467082
		END AS column_41, 
		CASE
			WHEN ((x22 < 62) OR (x22 IS NULL))
			AND ((x9 < 2) OR (x9 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
		THEN -0.0973824859
			WHEN ((x22 < 62) OR (x22 IS NULL))
			AND ((x9 < 2) OR (x9 IS NULL))
			AND (x3 >= 2)
		THEN 0.0347493403
			WHEN ((x22 < 62) OR (x22 IS NULL))
			AND (x9 >= 2)
			AND ((x31 < 226) OR (x31 IS NULL))
		THEN -0.0688317642
			WHEN ((x22 < 62) OR (x22 IS NULL))
			AND (x9 >= 2)
			AND (x31 >= 226)
		THEN 0.155878454
			WHEN (x22 >= 62)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x28 < 0.0350000001) OR (x28 IS NULL))
		THEN 0.0268387254
			WHEN (x22 >= 62)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x28 >= 0.0350000001)
		THEN 0.105266571
			WHEN (x22 >= 62)
			AND (x5 >= 2)
		THEN -0.0898320824
		END AS column_42, 
		CASE
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x9 < 2) OR (x9 IS NULL))
		THEN -0.0982126072
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x9 >= 2)
		THEN 0.0444651656
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND (x3 >= 2)
			AND ((x11 < 1) OR (x11 IS NULL))
		THEN 0.0969860479
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND (x3 >= 2)
			AND (x11 >= 1)
		THEN -0.0580039695
			WHEN (x1 >= 2)
			AND ((x31 < 12) OR (x31 IS NULL))
		THEN 0.526300788
			WHEN (x1 >= 2)
			AND (x31 >= 12)
			AND ((x4 < 352) OR (x4 IS NULL))
		THEN -0.0898615047
			WHEN (x1 >= 2)
			AND (x31 >= 12)
			AND (x4 >= 352)
		THEN 0.102866232
		END AS column_43, 
		CASE
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x9 < 2) OR (x9 IS NULL))
		THEN -0.0978162289
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x9 >= 2)
		THEN 0.0398143865
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND (x3 >= 2)
			AND ((x31 < 254) OR (x31 IS NULL))
		THEN -0.0585023239
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND (x3 >= 2)
			AND (x31 >= 254)
		THEN 0.0943677425
			WHEN (x1 >= 2)
			AND ((x36 < 0.0649999976) OR (x36 IS NULL))
			AND ((x22 < 49) OR (x22 IS NULL))
		THEN -0.0896561071
			WHEN (x1 >= 2)
			AND ((x36 < 0.0649999976) OR (x36 IS NULL))
			AND (x22 >= 49)
		THEN 0.102822922
			WHEN (x1 >= 2)
			AND (x36 >= 0.0649999976)
		THEN 0.529716432
		END AS column_44, 
		CASE
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x9 < 2) OR (x9 IS NULL))
		THEN -0.0973815545
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x9 >= 2)
		THEN 0.0356983095
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND (x3 >= 2)
			AND ((x32 < 62) OR (x32 IS NULL))
		THEN 0.0906246305
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND (x3 >= 2)
			AND (x32 >= 62)
		THEN -0.0562255792
			WHEN (x1 >= 2)
			AND ((x31 < 3) OR (x31 IS NULL))
		THEN 0.471500635
			WHEN (x1 >= 2)
			AND (x31 >= 3)
			AND ((x23 < 31) OR (x23 IS NULL))
		THEN -0.0890770778
			WHEN (x1 >= 2)
			AND (x31 >= 3)
			AND (x23 >= 31)
		THEN 0.102770165
		END AS column_45, 
		CASE
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x9 < 2) OR (x9 IS NULL))
		THEN -0.0969017074
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x9 >= 2)
		THEN 0.0320473425
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND (x3 >= 2)
			AND ((x31 < 254) OR (x31 IS NULL))
		THEN -0.0569002107
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND (x3 >= 2)
			AND (x31 >= 254)
		THEN 0.0876587927
			WHEN (x1 >= 2)
			AND ((x4 < 19) OR (x4 IS NULL))
		THEN 0.434839338
			WHEN (x1 >= 2)
			AND (x4 >= 19)
			AND ((x4 < 352) OR (x4 IS NULL))
		THEN -0.0883978009
			WHEN (x1 >= 2)
			AND (x4 >= 19)
			AND (x4 >= 352)
		THEN 0.102691166
		END AS column_46, 
		CASE
			WHEN ((x22 < 50) OR (x22 IS NULL))
			AND ((x4 < 29) OR (x4 IS NULL))
			AND ((x1 < 1) OR (x1 IS NULL))
		THEN -0.0642748997
			WHEN ((x22 < 50) OR (x22 IS NULL))
			AND ((x4 < 29) OR (x4 IS NULL))
			AND (x1 >= 1)
		THEN 0.39166674
			WHEN ((x22 < 50) OR (x22 IS NULL))
			AND (x4 >= 29)
			AND ((x4 < 53417) OR (x4 IS NULL))
		THEN -0.0987785608
			WHEN ((x22 < 50) OR (x22 IS NULL))
			AND (x4 >= 29)
			AND (x4 >= 53417)
		THEN 0.0750540495
			WHEN (x22 >= 50)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x22 < 79) OR (x22 IS NULL))
		THEN 0.198648125
			WHEN (x22 >= 50)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x22 >= 79)
		THEN 0.0801173672
			WHEN (x22 >= 50)
			AND (x5 >= 2)
		THEN -0.0870046467
		END AS column_47, 
		CASE
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x9 < 2) OR (x9 IS NULL))
		THEN -0.096061945
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x9 >= 2)
		THEN 0.0333772823
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND (x3 >= 2)
			AND ((x11 < 1) OR (x11 IS NULL))
		THEN 0.0884366259
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND (x3 >= 2)
			AND (x11 >= 1)
		THEN -0.0550670885
			WHEN (x1 >= 2)
			AND ((x4 < 19) OR (x4 IS NULL))
		THEN 0.279053867
			WHEN (x1 >= 2)
			AND (x4 >= 19)
			AND ((x4 < 352) OR (x4 IS NULL))
		THEN -0.0862019286
			WHEN (x1 >= 2)
			AND (x4 >= 19)
			AND (x4 >= 352)
		THEN 0.102236681
		END AS column_48, 
		CASE
			WHEN ((x22 < 50) OR (x22 IS NULL))
			AND ((x4 < 29) OR (x4 IS NULL))
			AND ((x1 < 1) OR (x1 IS NULL))
		THEN -0.0594804958
			WHEN ((x22 < 50) OR (x22 IS NULL))
			AND ((x4 < 29) OR (x4 IS NULL))
			AND (x1 >= 1)
		THEN 0.277613729
			WHEN ((x22 < 50) OR (x22 IS NULL))
			AND (x4 >= 29)
			AND ((x4 < 53417) OR (x4 IS NULL))
		THEN -0.0983302221
			WHEN ((x22 < 50) OR (x22 IS NULL))
			AND (x4 >= 29)
			AND (x4 >= 53417)
		THEN 0.0711990669
			WHEN (x22 >= 50)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x37 < 0.539999962) OR (x37 IS NULL))
		THEN 0.11716225
			WHEN (x22 >= 50)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x37 >= 0.539999962)
		THEN 0.0291077588
			WHEN (x22 >= 50)
			AND (x5 >= 2)
		THEN -0.0846700892
		END AS column_49, 
		CASE
			WHEN ((x22 < 50) OR (x22 IS NULL))
			AND ((x4 < 29) OR (x4 IS NULL))
			AND ((x2 < 2) OR (x2 IS NULL))
		THEN -0.0960959569
			WHEN ((x22 < 50) OR (x22 IS NULL))
			AND ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
		THEN 0.120591819
			WHEN ((x22 < 50) OR (x22 IS NULL))
			AND (x4 >= 29)
			AND ((x9 < 1) OR (x9 IS NULL))
		THEN -0.0989833996
			WHEN ((x22 < 50) OR (x22 IS NULL))
			AND (x4 >= 29)
			AND (x9 >= 1)
		THEN 0.0291408189
			WHEN (x22 >= 50)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND ((x28 < 0.0350000001) OR (x28 IS NULL))
		THEN 0.0104794558
			WHEN (x22 >= 50)
			AND ((x5 < 2) OR (x5 IS NULL))
			AND (x28 >= 0.0350000001)
		THEN 0.112431861
			WHEN (x22 >= 50)
			AND (x5 >= 2)
		THEN -0.0834940597
		END AS column_50, 
		CASE
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND ((x9 < 2) OR (x9 IS NULL))
		THEN -0.0947184786
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND ((x3 < 2) OR (x3 IS NULL))
			AND (x9 >= 2)
		THEN 0.0317576341
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND (x3 >= 2)
			AND ((x32 < 21) OR (x32 IS NULL))
		THEN 0.0905334204
			WHEN ((x1 < 2) OR (x1 IS NULL))
			AND (x3 >= 2)
			AND (x32 >= 21)
		THEN -0.0375691466
			WHEN (x1 >= 2)
			AND ((x23 < 5) OR (x23 IS NULL))
			AND ((x36 < 0.234999999) OR (x36 IS NULL))
		THEN -0.0802951157
			WHEN (x1 >= 2)
			AND ((x23 < 5) OR (x23 IS NULL))
			AND (x36 >= 0.234999999)
		THEN 0.0884586275
			WHEN (x1 >= 2)
			AND (x23 >= 5)
			AND ((x4 < 19) OR (x4 IS NULL))
		THEN 0.188892394
			WHEN (x1 >= 2)
			AND (x23 >= 5)
			AND (x4 >= 19)
		THEN 0.0909775868
		END AS column_51, 
		CASE
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND ((x22 < 22) OR (x22 IS NULL))
			AND ((x35 < 0.995000005) OR (x35 IS NULL))
		THEN -0.0867452472
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND ((x22 < 22) OR (x22 IS NULL))
			AND (x35 >= 0.995000005)
		THEN 0.0605996922
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND (x22 >= 22)
			AND ((x22 < 79) OR (x22 IS NULL))
		THEN 0.230706766
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND (x22 >= 22)
			AND (x22 >= 79)
		THEN 0.071079351
			WHEN (x5 >= 1)
			AND ((x4 < 52200) OR (x4 IS NULL))
			AND ((x9 < 18) OR (x9 IS NULL))
		THEN -0.0999394506
			WHEN (x5 >= 1)
			AND ((x4 < 52200) OR (x4 IS NULL))
			AND (x9 >= 18)
		THEN 0.0930004641
			WHEN (x5 >= 1)
			AND (x4 >= 52200)
		THEN 0.10096284
		END AS column_52, 
		CASE
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND ((x23 < 8) OR (x23 IS NULL))
			AND ((x33 < 0.0350000001) OR (x33 IS NULL))
		THEN 0.0538855009
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND ((x23 < 8) OR (x23 IS NULL))
			AND (x33 >= 0.0350000001)
		THEN -0.0831160396
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND (x23 >= 8)
			AND ((x4 < 1116) OR (x4 IS NULL))
		THEN 0.134934589
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND (x23 >= 8)
			AND (x4 >= 1116)
		THEN -0.0721504018
			WHEN (x5 >= 1)
			AND ((x4 < 52200) OR (x4 IS NULL))
			AND ((x9 < 19) OR (x9 IS NULL))
		THEN -0.0998651609
			WHEN (x5 >= 1)
			AND ((x4 < 52200) OR (x4 IS NULL))
			AND (x9 >= 19)
		THEN 0.096574299
			WHEN (x5 >= 1)
			AND (x4 >= 52200)
		THEN 0.0962309092
		END AS column_53, 
		CASE
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND ((x2 < 2) OR (x2 IS NULL))
		THEN -0.0943369046
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.172579616
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND (x5 >= 2)
		THEN -0.0797179043
			WHEN (x4 >= 29)
			AND ((x22 < 433) OR (x22 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
		THEN -0.0984434932
			WHEN (x4 >= 29)
			AND ((x22 < 433) OR (x22 IS NULL))
			AND (x9 >= 1)
		THEN 0.0258659516
			WHEN (x4 >= 29)
			AND (x22 >= 433)
		THEN 0.0839205608
		END AS column_54, 
		CASE
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND ((x23 < 8) OR (x23 IS NULL))
			AND ((x33 < 0.0350000001) OR (x33 IS NULL))
		THEN 0.0478310287
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND ((x23 < 8) OR (x23 IS NULL))
			AND (x33 >= 0.0350000001)
		THEN -0.0803462788
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND (x23 >= 8)
			AND ((x11 < 1) OR (x11 IS NULL))
		THEN 0.131962106
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND (x23 >= 8)
			AND (x11 >= 1)
		THEN -0.069232136
			WHEN (x5 >= 1)
			AND ((x4 < 52200) OR (x4 IS NULL))
			AND ((x9 < 19) OR (x9 IS NULL))
		THEN -0.0997183621
			WHEN (x5 >= 1)
			AND ((x4 < 52200) OR (x4 IS NULL))
			AND (x9 >= 19)
		THEN 0.0845487416
			WHEN (x5 >= 1)
			AND (x4 >= 52200)
		THEN 0.0914993733
		END AS column_55, 
		CASE
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND ((x2 < 2) OR (x2 IS NULL))
		THEN -0.0932399035
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.165376648
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND (x5 >= 2)
		THEN -0.0764688402
			WHEN (x4 >= 29)
			AND ((x22 < 433) OR (x22 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
		THEN -0.0979964584
			WHEN (x4 >= 29)
			AND ((x22 < 433) OR (x22 IS NULL))
			AND (x9 >= 1)
		THEN 0.0241283067
			WHEN (x4 >= 29)
			AND (x22 >= 433)
		THEN 0.0807280466
		END AS column_56, 
		CASE
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND ((x28 < 0.305000007) OR (x28 IS NULL))
			AND ((x2 < 3) OR (x2 IS NULL))
		THEN 0.1160798
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND ((x28 < 0.305000007) OR (x28 IS NULL))
			AND (x2 >= 3)
		THEN -0.144972727
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND (x28 >= 0.305000007)
			AND ((x9 < 1) OR (x9 IS NULL))
		THEN -0.0959918797
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND (x28 >= 0.305000007)
			AND (x9 >= 1)
		THEN 0.0242749155
			WHEN (x35 >= 0.995000005)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND ((x2 < 1) OR (x2 IS NULL))
		THEN -0.0677275136
			WHEN (x35 >= 0.995000005)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND (x2 >= 1)
		THEN 0.115404658
			WHEN (x35 >= 0.995000005)
			AND (x5 >= 1)
		THEN -0.0859638005
		END AS column_57, 
		CASE
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND ((x2 < 2) OR (x2 IS NULL))
		THEN -0.0918829516
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.158682331
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND (x5 >= 2)
		THEN -0.0730944797
			WHEN (x4 >= 29)
			AND ((x22 < 433) OR (x22 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
		THEN -0.0974977538
			WHEN (x4 >= 29)
			AND ((x22 < 433) OR (x22 IS NULL))
			AND (x9 >= 1)
		THEN 0.0197776388
			WHEN (x4 >= 29)
			AND (x22 >= 433)
		THEN 0.0774173066
		END AS column_58, 
		CASE
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND ((x4 < 29) OR (x4 IS NULL))
		THEN -0.0162640139
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND ((x4 < 53417) OR (x4 IS NULL))
			AND (x4 >= 29)
		THEN -0.0978395194
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND (x4 >= 53417)
		THEN 0.0676227435
			WHEN (x35 >= 0.995000005)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND ((x2 < 1) OR (x2 IS NULL))
		THEN -0.0641066432
			WHEN (x35 >= 0.995000005)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND (x2 >= 1)
		THEN 0.108347714
			WHEN (x35 >= 0.995000005)
			AND (x5 >= 1)
		THEN -0.0835934505
		END AS column_59, 
		CASE
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND ((x2 < 2) OR (x2 IS NULL))
		THEN -0.0909136459
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.149259195
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND (x5 >= 2)
		THEN -0.0712211356
			WHEN (x4 >= 29)
			AND ((x4 < 1032) OR (x4 IS NULL))
			AND ((x36 < 0.125) OR (x36 IS NULL))
		THEN -0.0990853831
			WHEN (x4 >= 29)
			AND ((x4 < 1032) OR (x4 IS NULL))
			AND (x36 >= 0.125)
		THEN 0.0458532423
			WHEN (x4 >= 29)
			AND (x4 >= 1032)
			AND ((x31 < 255) OR (x31 IS NULL))
		THEN -0.0835605189
			WHEN (x4 >= 29)
			AND (x4 >= 1032)
			AND (x31 >= 255)
		THEN 0.115653455
		END AS column_60, 
		CASE
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
			AND ((x4 < 29) OR (x4 IS NULL))
		THEN -0.0159517303
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
			AND (x4 >= 29)
		THEN -0.100014918
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND (x9 >= 1)
			AND ((x4 < 1043) OR (x4 IS NULL))
		THEN -0.075302802
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND (x9 >= 1)
			AND (x4 >= 1043)
		THEN 0.114122525
			WHEN (x35 >= 0.995000005)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND ((x2 < 1) OR (x2 IS NULL))
		THEN -0.0604400933
			WHEN (x35 >= 0.995000005)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND (x2 >= 1)
		THEN 0.10147772
			WHEN (x35 >= 0.995000005)
			AND (x5 >= 1)
		THEN -0.080940105
		END AS column_61, 
		CASE
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND ((x2 < 2) OR (x2 IS NULL))
		THEN -0.0898737088
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND ((x5 < 2) OR (x5 IS NULL))
		THEN 0.14014183
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND (x5 >= 2)
		THEN -0.0693285391
			WHEN (x4 >= 29)
			AND ((x4 < 1032) OR (x4 IS NULL))
			AND ((x36 < 0.125) OR (x36 IS NULL))
		THEN -0.0988136753
			WHEN (x4 >= 29)
			AND ((x4 < 1032) OR (x4 IS NULL))
			AND (x36 >= 0.125)
		THEN 0.0479810946
			WHEN (x4 >= 29)
			AND (x4 >= 1032)
			AND ((x31 < 255) OR (x31 IS NULL))
		THEN -0.0811305195
			WHEN (x4 >= 29)
			AND (x4 >= 1032)
			AND (x31 >= 255)
		THEN 0.11218939
		END AS column_62, 
		CASE
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND ((x23 < 8) OR (x23 IS NULL))
			AND ((x35 < 0.88499999) OR (x35 IS NULL))
		THEN -0.0819303691
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND ((x23 < 8) OR (x23 IS NULL))
			AND (x35 >= 0.88499999)
		THEN 0.0355261452
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND (x23 >= 8)
			AND ((x4 < 1116) OR (x4 IS NULL))
		THEN 0.130410299
			WHEN ((x5 < 1) OR (x5 IS NULL))
			AND (x23 >= 8)
			AND (x4 >= 1116)
		THEN -0.0545510352
			WHEN (x5 >= 1)
			AND ((x9 < 1) OR (x9 IS NULL))
		THEN -0.0989266783
			WHEN (x5 >= 1)
			AND (x9 >= 1)
			AND ((x31 < 253) OR (x31 IS NULL))
		THEN -0.0587306134
			WHEN (x5 >= 1)
			AND (x9 >= 1)
			AND (x31 >= 253)
		THEN 0.132995442
		END AS column_63, 
		CASE
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND ((x2 < 2) OR (x2 IS NULL))
		THEN -0.0883942097
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
		THEN 0.186736435
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND (x2 >= 3)
		THEN 0.00915266294
			WHEN (x4 >= 29)
			AND ((x4 < 1032) OR (x4 IS NULL))
			AND ((x36 < 0.114999995) OR (x36 IS NULL))
		THEN -0.0985402316
			WHEN (x4 >= 29)
			AND ((x4 < 1032) OR (x4 IS NULL))
			AND (x36 >= 0.114999995)
		THEN 0.0444673412
			WHEN (x4 >= 29)
			AND (x4 >= 1032)
			AND ((x31 < 255) OR (x31 IS NULL))
		THEN -0.0773139223
			WHEN (x4 >= 29)
			AND (x4 >= 1032)
			AND (x31 >= 255)
		THEN 0.108003102
		END AS column_64, 
		CASE
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND ((x9 < 2) OR (x9 IS NULL))
			AND ((x7 < 2) OR (x7 IS NULL))
		THEN -0.0906694233
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND ((x9 < 2) OR (x9 IS NULL))
			AND (x7 >= 2)
		THEN 0.116316274
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND (x9 >= 2)
			AND ((x34 < 0.0249999985) OR (x34 IS NULL))
		THEN 0.121016078
			WHEN ((x35 < 0.995000005) OR (x35 IS NULL))
			AND (x9 >= 2)
			AND (x34 >= 0.0249999985)
		THEN -0.0679601058
			WHEN (x35 >= 0.995000005)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND ((x2 < 1) OR (x2 IS NULL))
		THEN -0.0555587411
			WHEN (x35 >= 0.995000005)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND (x2 >= 1)
		THEN 0.0953614488
			WHEN (x35 >= 0.995000005)
			AND (x5 >= 1)
		THEN -0.0746263489
		END AS column_65, 
		CASE
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND ((x2 < 2) OR (x2 IS NULL))
		THEN -0.0864328742
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
		THEN 0.181242749
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND (x2 >= 3)
		THEN 0.00970795751
			WHEN (x4 >= 29)
			AND ((x4 < 1032) OR (x4 IS NULL))
			AND ((x36 < 0.114999995) OR (x36 IS NULL))
		THEN -0.0981742144
			WHEN (x4 >= 29)
			AND ((x4 < 1032) OR (x4 IS NULL))
			AND (x36 >= 0.114999995)
		THEN 0.045828782
			WHEN (x4 >= 29)
			AND (x4 >= 1032)
			AND ((x31 < 255) OR (x31 IS NULL))
		THEN -0.074300997
			WHEN (x4 >= 29)
			AND (x4 >= 1032)
			AND (x31 >= 255)
		THEN 0.104851164
		END AS column_66, 
		CASE
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND ((x2 < 2) OR (x2 IS NULL))
		THEN -0.0853545219
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
		THEN 0.179894611
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND (x2 >= 3)
		THEN 0.00885615405
			WHEN (x4 >= 29)
			AND ((x4 < 1032) OR (x4 IS NULL))
			AND ((x36 < 0.125) OR (x36 IS NULL))
		THEN -0.0977589339
			WHEN (x4 >= 29)
			AND ((x4 < 1032) OR (x4 IS NULL))
			AND (x36 >= 0.125)
		THEN 0.0478438474
			WHEN (x4 >= 29)
			AND (x4 >= 1032)
			AND ((x31 < 255) OR (x31 IS NULL))
		THEN -0.0719865486
			WHEN (x4 >= 29)
			AND (x4 >= 1032)
			AND (x31 >= 255)
		THEN 0.0979670435
		END AS column_67, 
		CASE
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
			AND ((x7 < 2) OR (x7 IS NULL))
		THEN -0.0927413404
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
			AND (x7 >= 2)
		THEN 0.0922201052
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 1)
			AND ((x34 < 0.0249999985) OR (x34 IS NULL))
		THEN 0.110860303
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 1)
			AND (x34 >= 0.0249999985)
		THEN -0.0648760647
			WHEN (x35 >= 0.704999983)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND ((x4 < 205) OR (x4 IS NULL))
		THEN 0.0338759758
			WHEN (x35 >= 0.704999983)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND (x4 >= 205)
		THEN 0.177774653
			WHEN (x35 >= 0.704999983)
			AND (x5 >= 1)
		THEN -0.07929869
		END AS column_68, 
		CASE
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND ((x2 < 2) OR (x2 IS NULL))
		THEN -0.0832657441
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
		THEN 0.175213918
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND (x2 >= 3)
		THEN 0.0114661856
			WHEN (x4 >= 29)
			AND ((x9 < 2) OR (x9 IS NULL))
			AND ((x35 < 0.88499999) OR (x35 IS NULL))
		THEN -0.100961365
			WHEN (x4 >= 29)
			AND ((x9 < 2) OR (x9 IS NULL))
			AND (x35 >= 0.88499999)
		THEN -0.00933837518
			WHEN (x4 >= 29)
			AND (x9 >= 2)
			AND ((x31 < 253) OR (x31 IS NULL))
		THEN -0.0444267727
			WHEN (x4 >= 29)
			AND (x9 >= 2)
			AND (x31 >= 253)
		THEN 0.132908374
		END AS column_69, 
		CASE
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
			AND ((x4 < 29) OR (x4 IS NULL))
		THEN -0.018647803
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
			AND (x4 >= 29)
		THEN -0.101708353
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 1)
			AND ((x4 < 1043) OR (x4 IS NULL))
		THEN -0.0641669109
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 1)
			AND (x4 >= 1043)
		THEN 0.109259263
			WHEN (x35 >= 0.704999983)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND ((x4 < 205) OR (x4 IS NULL))
		THEN 0.0309675876
			WHEN (x35 >= 0.704999983)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND (x4 >= 205)
		THEN 0.181161761
			WHEN (x35 >= 0.704999983)
			AND (x5 >= 1)
		THEN -0.0776389539
		END AS column_70, 
		CASE
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND ((x2 < 2) OR (x2 IS NULL))
		THEN -0.0819110572
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND ((x35 < 0.105000004) OR (x35 IS NULL))
		THEN -0.0048859599
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND (x35 >= 0.105000004)
		THEN 0.155805662
			WHEN (x4 >= 29)
			AND ((x4 < 1032) OR (x4 IS NULL))
			AND ((x36 < 0.0949999988) OR (x36 IS NULL))
		THEN -0.0971968397
			WHEN (x4 >= 29)
			AND ((x4 < 1032) OR (x4 IS NULL))
			AND (x36 >= 0.0949999988)
		THEN 0.0453855693
			WHEN (x4 >= 29)
			AND (x4 >= 1032)
			AND ((x34 < 0.0249999985) OR (x34 IS NULL))
		THEN 0.0873659253
			WHEN (x4 >= 29)
			AND (x4 >= 1032)
			AND (x34 >= 0.0249999985)
		THEN -0.0817384049
		END AS column_71, 
		CASE
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
			AND ((x4 < 29) OR (x4 IS NULL))
		THEN -0.0168870836
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
			AND (x4 >= 29)
		THEN -0.10188587
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 1)
			AND ((x34 < 0.0249999985) OR (x34 IS NULL))
		THEN 0.104649626
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 1)
			AND (x34 >= 0.0249999985)
		THEN -0.0610994361
			WHEN (x35 >= 0.704999983)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND ((x4 < 205) OR (x4 IS NULL))
		THEN 0.0283974949
			WHEN (x35 >= 0.704999983)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND (x4 >= 205)
		THEN 0.18346253
			WHEN (x35 >= 0.704999983)
			AND (x5 >= 1)
		THEN -0.074752681
		END AS column_72, 
		CASE
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND ((x2 < 2) OR (x2 IS NULL))
		THEN -0.0805134103
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
		THEN 0.169046342
			WHEN ((x4 < 29) OR (x4 IS NULL))
			AND (x2 >= 2)
			AND (x2 >= 3)
		THEN 0.0029260132
			WHEN (x4 >= 29)
			AND ((x4 < 1032) OR (x4 IS NULL))
			AND ((x36 < 0.0949999988) OR (x36 IS NULL))
		THEN -0.0967052206
			WHEN (x4 >= 29)
			AND ((x4 < 1032) OR (x4 IS NULL))
			AND (x36 >= 0.0949999988)
		THEN 0.0471190475
			WHEN (x4 >= 29)
			AND (x4 >= 1032)
			AND ((x31 < 255) OR (x31 IS NULL))
		THEN -0.0653548688
			WHEN (x4 >= 29)
			AND (x4 >= 1032)
			AND (x31 >= 255)
		THEN 0.100732863
		END AS column_73, 
		CASE
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 2) OR (x9 IS NULL))
			AND ((x3 < 1) OR (x3 IS NULL))
		THEN -0.0981689692
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 2) OR (x9 IS NULL))
			AND (x3 >= 1)
		THEN -0.00626459112
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 2)
			AND ((x34 < 0.0249999985) OR (x34 IS NULL))
		THEN 0.109619834
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 2)
			AND (x34 >= 0.0249999985)
		THEN -0.0568280332
			WHEN (x35 >= 0.704999983)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND ((x4 < 205) OR (x4 IS NULL))
		THEN 0.0298434142
			WHEN (x35 >= 0.704999983)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND (x4 >= 205)
		THEN 0.184419021
			WHEN (x35 >= 0.704999983)
			AND (x5 >= 1)
		THEN -0.0715676174
		END AS column_74, 
		CASE
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 19) OR (x9 IS NULL))
			AND ((x39 < 0.0249999985) OR (x39 IS NULL))
		THEN -0.0936880633
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 19) OR (x9 IS NULL))
			AND (x39 >= 0.0249999985)
		THEN -0.000327543472
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 19)
		THEN 0.0914269015
			WHEN (x35 >= 0.704999983)
			AND ((x4 < 19) OR (x4 IS NULL))
			AND ((x31 < 2) OR (x31 IS NULL))
		THEN 0.0247673225
			WHEN (x35 >= 0.704999983)
			AND ((x4 < 19) OR (x4 IS NULL))
			AND (x31 >= 2)
		THEN 0.186894506
			WHEN (x35 >= 0.704999983)
			AND (x4 >= 19)
			AND ((x4 < 207) OR (x4 IS NULL))
		THEN -0.0843131095
			WHEN (x35 >= 0.704999983)
			AND (x4 >= 19)
			AND (x4 >= 207)
		THEN 0.143973991
		END AS column_75, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 12634) OR (x4 IS NULL))
		THEN -0.0958995894
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 12634)
		THEN 0.0493267737
			WHEN (x2 >= 2)
			AND ((x4 < 9) OR (x4 IS NULL))
			AND ((x35 < 0.129999995) OR (x35 IS NULL))
		THEN 0.0340689197
			WHEN (x2 >= 2)
			AND ((x4 < 9) OR (x4 IS NULL))
			AND (x35 >= 0.129999995)
		THEN 0.18890591
			WHEN (x2 >= 2)
			AND (x4 >= 9)
			AND ((x4 < 205) OR (x4 IS NULL))
		THEN -0.0782649443
			WHEN (x2 >= 2)
			AND (x4 >= 9)
			AND (x4 >= 205)
		THEN 0.0449915715
		END AS column_76, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 11516) OR (x4 IS NULL))
		THEN -0.0954787582
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 11516)
		THEN 0.045219522
			WHEN (x2 >= 2)
			AND ((x4 < 9) OR (x4 IS NULL))
			AND ((x35 < 0.129999995) OR (x35 IS NULL))
		THEN 0.031642139
			WHEN (x2 >= 2)
			AND ((x4 < 9) OR (x4 IS NULL))
			AND (x35 >= 0.129999995)
		THEN 0.178613961
			WHEN (x2 >= 2)
			AND (x4 >= 9)
			AND ((x34 < 0.0350000001) OR (x34 IS NULL))
		THEN 0.0105616851
			WHEN (x2 >= 2)
			AND (x4 >= 9)
			AND (x34 >= 0.0350000001)
		THEN -0.102225043
		END AS column_77, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 6794) OR (x4 IS NULL))
		THEN -0.0950206965
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 6794)
		THEN 0.0415253304
			WHEN (x2 >= 2)
			AND ((x4 < 9) OR (x4 IS NULL))
			AND ((x35 < 0.129999995) OR (x35 IS NULL))
		THEN 0.0293647405
			WHEN (x2 >= 2)
			AND ((x4 < 9) OR (x4 IS NULL))
			AND (x35 >= 0.129999995)
		THEN 0.167792097
			WHEN (x2 >= 2)
			AND (x4 >= 9)
			AND ((x4 < 205) OR (x4 IS NULL))
		THEN -0.0742588267
			WHEN (x2 >= 2)
			AND (x4 >= 9)
			AND (x4 >= 205)
		THEN 0.0440433249
		END AS column_78, 
		CASE
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 20) OR (x9 IS NULL))
			AND ((x39 < 0.0249999985) OR (x39 IS NULL))
		THEN -0.0903335735
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 20) OR (x9 IS NULL))
			AND (x39 >= 0.0249999985)
		THEN 0.00945944246
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 20)
		THEN 0.0886507854
			WHEN (x35 >= 0.704999983)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND ((x4 < 205) OR (x4 IS NULL))
		THEN 0.0260954387
			WHEN (x35 >= 0.704999983)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND (x4 >= 205)
		THEN 0.176863804
			WHEN (x35 >= 0.704999983)
			AND (x5 >= 1)
		THEN -0.0642226562
		END AS column_79, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 3936) OR (x4 IS NULL))
		THEN -0.0941264182
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 3936)
		THEN 0.0375895984
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND ((x32 < 115) OR (x32 IS NULL))
		THEN 0.175259039
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND (x32 >= 115)
		THEN -0.0614125207
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND ((x34 < 0.0249999985) OR (x34 IS NULL))
		THEN 0.0423953012
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND (x34 >= 0.0249999985)
		THEN -0.106239878
		END AS column_80, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 3425) OR (x4 IS NULL))
		THEN -0.0935514346
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 3425)
		THEN 0.0347906835
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND ((x32 < 115) OR (x32 IS NULL))
		THEN 0.170248851
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND (x32 >= 115)
		THEN -0.0599467158
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND ((x34 < 0.0249999985) OR (x34 IS NULL))
		THEN 0.0378716625
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND (x34 >= 0.0249999985)
		THEN -0.104636692
		END AS column_81, 
		CASE
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 20) OR (x9 IS NULL))
			AND ((x39 < 0.0350000001) OR (x39 IS NULL))
		THEN -0.0889669433
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 20) OR (x9 IS NULL))
			AND (x39 >= 0.0350000001)
		THEN 0.011865898
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 20)
		THEN 0.087075524
			WHEN (x35 >= 0.704999983)
			AND ((x4 < 207) OR (x4 IS NULL))
			AND ((x4 < 19) OR (x4 IS NULL))
		THEN 0.0913191959
			WHEN (x35 >= 0.704999983)
			AND ((x4 < 207) OR (x4 IS NULL))
			AND (x4 >= 19)
		THEN -0.0785757229
			WHEN (x35 >= 0.704999983)
			AND (x4 >= 207)
		THEN 0.140499935
		END AS column_82, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 2960) OR (x4 IS NULL))
		THEN -0.0924875066
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 2960)
		THEN 0.0320749916
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND ((x32 < 115) OR (x32 IS NULL))
		THEN 0.162532568
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND (x32 >= 115)
		THEN -0.0563289337
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND ((x34 < 0.0249999985) OR (x34 IS NULL))
		THEN 0.0352499336
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND (x34 >= 0.0249999985)
		THEN -0.10228087
		END AS column_83, 
		CASE
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 20) OR (x9 IS NULL))
			AND ((x39 < 0.0249999985) OR (x39 IS NULL))
		THEN -0.087748304
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 20) OR (x9 IS NULL))
			AND (x39 >= 0.0249999985)
		THEN 0.0100008342
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 20)
		THEN 0.081800662
			WHEN (x35 >= 0.704999983)
			AND ((x4 < 207) OR (x4 IS NULL))
			AND ((x4 < 19) OR (x4 IS NULL))
		THEN 0.0817455947
			WHEN (x35 >= 0.704999983)
			AND ((x4 < 207) OR (x4 IS NULL))
			AND (x4 >= 19)
		THEN -0.0766325295
			WHEN (x35 >= 0.704999983)
			AND (x4 >= 207)
		THEN 0.131861061
		END AS column_84, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 2592) OR (x4 IS NULL))
		THEN -0.0912908465
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 2592)
		THEN 0.0291065425
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND ((x32 < 115) OR (x32 IS NULL))
		THEN 0.154950902
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND (x32 >= 115)
		THEN -0.0527647249
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND ((x34 < 0.0249999985) OR (x34 IS NULL))
		THEN 0.0326718055
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND (x34 >= 0.0249999985)
		THEN -0.0990519524
		END AS column_85, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 2515) OR (x4 IS NULL))
		THEN -0.090484865
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 2515)
		THEN 0.0270871669
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND ((x34 < 0.0149999997) OR (x34 IS NULL))
		THEN -0.038728565
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND (x34 >= 0.0149999997)
		THEN 0.158948407
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND ((x34 < 0.0249999985) OR (x34 IS NULL))
		THEN 0.0292855836
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND (x34 >= 0.0249999985)
		THEN -0.0959413573
		END AS column_86, 
		CASE
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
			AND ((x4 < 29) OR (x4 IS NULL))
		THEN -0.00178526959
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
			AND (x4 >= 29)
		THEN -0.10100875
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 1)
			AND ((x31 < 253) OR (x31 IS NULL))
		THEN -0.0360390469
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 1)
			AND (x31 >= 253)
		THEN 0.107681036
			WHEN (x35 >= 0.704999983)
			AND ((x4 < 207) OR (x4 IS NULL))
			AND ((x4 < 19) OR (x4 IS NULL))
		THEN 0.0716921762
			WHEN (x35 >= 0.704999983)
			AND ((x4 < 207) OR (x4 IS NULL))
			AND (x4 >= 19)
		THEN -0.07425455
			WHEN (x35 >= 0.704999983)
			AND (x4 >= 207)
		THEN 0.124921694
		END AS column_87, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 2076) OR (x4 IS NULL))
		THEN -0.0889077708
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 2076)
		THEN 0.0212073401
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND ((x34 < 0.0149999997) OR (x34 IS NULL))
		THEN -0.0346651115
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND (x34 >= 0.0149999997)
		THEN 0.153126523
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND ((x34 < 0.0249999985) OR (x34 IS NULL))
		THEN 0.0267052148
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND (x34 >= 0.0249999985)
		THEN -0.091885671
		END AS column_88, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 2048) OR (x4 IS NULL))
		THEN -0.0879352018
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 2048)
		THEN 0.0199110862
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND ((x34 < 0.0149999997) OR (x34 IS NULL))
		THEN -0.0332275666
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND (x34 >= 0.0149999997)
		THEN 0.148871765
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND ((x31 < 8) OR (x31 IS NULL))
		THEN 0.0795448124
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND (x31 >= 8)
		THEN -0.0502635501
		END AS column_89, 
		CASE
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 20) OR (x9 IS NULL))
			AND ((x39 < 0.0350000001) OR (x39 IS NULL))
		THEN -0.0834080428
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 20) OR (x9 IS NULL))
			AND (x39 >= 0.0350000001)
		THEN 0.0156119866
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 20)
		THEN 0.0816486478
			WHEN (x35 >= 0.704999983)
			AND ((x4 < 207) OR (x4 IS NULL))
			AND ((x4 < 19) OR (x4 IS NULL))
		THEN 0.0620674454
			WHEN (x35 >= 0.704999983)
			AND ((x4 < 207) OR (x4 IS NULL))
			AND (x4 >= 19)
		THEN -0.0717672333
			WHEN (x35 >= 0.704999983)
			AND (x4 >= 207)
		THEN 0.116558552
		END AS column_90, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 1960) OR (x4 IS NULL))
		THEN -0.0862291679
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 1960)
		THEN 0.0183119811
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND ((x34 < 0.0149999997) OR (x34 IS NULL))
		THEN -0.028972974
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND (x34 >= 0.0149999997)
		THEN 0.143612668
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND ((x34 < 0.0249999985) OR (x34 IS NULL))
		THEN 0.0260824542
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND (x34 >= 0.0249999985)
		THEN -0.084352985
		END AS column_91, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 1939) OR (x4 IS NULL))
		THEN -0.0850814134
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 1939)
		THEN 0.0172057841
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND ((x34 < 0.0149999997) OR (x34 IS NULL))
		THEN -0.0277474318
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND (x34 >= 0.0149999997)
		THEN 0.139668211
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND ((x34 < 0.0249999985) OR (x34 IS NULL))
		THEN 0.0234548878
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND (x34 >= 0.0249999985)
		THEN -0.0800538808
		END AS column_92, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 1919) OR (x4 IS NULL))
		THEN -0.0838662535
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 1919)
		THEN 0.0161856338
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND ((x34 < 0.0149999997) OR (x34 IS NULL))
		THEN -0.0265705585
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND (x34 >= 0.0149999997)
		THEN 0.135771334
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND ((x31 < 8) OR (x31 IS NULL))
		THEN 0.0674548075
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND (x31 >= 8)
		THEN -0.042252522
		END AS column_93, 
		CASE
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
			AND ((x29 < 0.075000003) OR (x29 IS NULL))
		THEN -0.0793628916
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
			AND (x29 >= 0.075000003)
		THEN 0.0598579347
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 1)
			AND ((x31 < 253) OR (x31 IS NULL))
		THEN -0.0301163048
			WHEN ((x35 < 0.704999983) OR (x35 IS NULL))
			AND (x9 >= 1)
			AND (x31 >= 253)
		THEN 0.105042554
			WHEN (x35 >= 0.704999983)
			AND ((x4 < 207) OR (x4 IS NULL))
			AND ((x4 < 19) OR (x4 IS NULL))
		THEN 0.0530291758
			WHEN (x35 >= 0.704999983)
			AND ((x4 < 207) OR (x4 IS NULL))
			AND (x4 >= 19)
		THEN -0.0692647174
			WHEN (x35 >= 0.704999983)
			AND (x4 >= 207)
		THEN 0.108368747
		END AS column_94, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 1649) OR (x4 IS NULL))
		THEN -0.0814201459
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 1649)
		THEN 0.0119226268
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND ((x34 < 0.0149999997) OR (x34 IS NULL))
		THEN -0.0227766391
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND (x34 >= 0.0149999997)
		THEN 0.131034061
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND ((x34 < 0.0249999985) OR (x34 IS NULL))
		THEN 0.0222345255
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND (x34 >= 0.0249999985)
		THEN -0.0723752305
		END AS column_95, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x4 < 1644) OR (x4 IS NULL))
		THEN -0.0800172538
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x4 >= 1644)
		THEN 0.0112721454
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND ((x34 < 0.0149999997) OR (x34 IS NULL))
		THEN -0.0218009818
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND (x34 >= 0.0149999997)
		THEN 0.127280146
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND ((x2 < 9) OR (x2 IS NULL))
		THEN -0.0493582115
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND (x2 >= 9)
		THEN 0.035847906
		END AS column_96, 
		CASE
			WHEN ((x35 < 0.105000004) OR (x35 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
			AND ((x4 < 1) OR (x4 IS NULL))
		THEN 0.0181304272
			WHEN ((x35 < 0.105000004) OR (x35 IS NULL))
			AND ((x9 < 1) OR (x9 IS NULL))
			AND (x4 >= 1)
		THEN -0.107545577
			WHEN ((x35 < 0.105000004) OR (x35 IS NULL))
			AND (x9 >= 1)
			AND ((x0 < 21) OR (x0 IS NULL))
		THEN 0.107153825
			WHEN ((x35 < 0.105000004) OR (x35 IS NULL))
			AND (x9 >= 1)
			AND (x0 >= 21)
		THEN -0.0200515836
			WHEN (x35 >= 0.105000004)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND ((x22 < 22) OR (x22 IS NULL))
		THEN 0.0205345526
			WHEN (x35 >= 0.105000004)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND (x22 >= 22)
		THEN 0.106038831
			WHEN (x35 >= 0.105000004)
			AND (x5 >= 1)
		THEN -0.0577011295
		END AS column_97, 
		CASE
			WHEN ((x35 < 0.105000004) OR (x35 IS NULL))
			AND ((x4 < 1043) OR (x4 IS NULL))
			AND ((x4 < 1) OR (x4 IS NULL))
		THEN 0.0165940821
			WHEN ((x35 < 0.105000004) OR (x35 IS NULL))
			AND ((x4 < 1043) OR (x4 IS NULL))
			AND (x4 >= 1)
		THEN -0.106900923
			WHEN ((x35 < 0.105000004) OR (x35 IS NULL))
			AND (x4 >= 1043)
			AND ((x31 < 255) OR (x31 IS NULL))
		THEN -0.0307159461
			WHEN ((x35 < 0.105000004) OR (x35 IS NULL))
			AND (x4 >= 1043)
			AND (x31 >= 255)
		THEN 0.100243948
			WHEN (x35 >= 0.105000004)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND ((x4 < 205) OR (x4 IS NULL))
		THEN 0.0150276888
			WHEN (x35 >= 0.105000004)
			AND ((x5 < 1) OR (x5 IS NULL))
			AND (x4 >= 205)
		THEN 0.0932338387
			WHEN (x35 >= 0.105000004)
			AND (x5 >= 1)
		THEN -0.0562581234
		END AS column_98, 
		CASE
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND ((x5 < 5482) OR (x5 IS NULL))
		THEN -0.0758788213
			WHEN ((x2 < 2) OR (x2 IS NULL))
			AND (x5 >= 5482)
		THEN 0.00267696078
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND ((x35 < 0.105000004) OR (x35 IS NULL))
		THEN -0.00956515875
			WHEN (x2 >= 2)
			AND ((x2 < 3) OR (x2 IS NULL))
			AND (x35 >= 0.105000004)
		THEN 0.12478818
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND ((x36 < 0.125) OR (x36 IS NULL))
		THEN -0.0316929705
			WHEN (x2 >= 2)
			AND (x2 >= 3)
			AND (x36 >= 0.125)
		THEN 0.0678183138
		END AS column_99
	FROM clean_kdd;
"""

test = pd.read_sql(query, conn)

To prevent numerical overflow, I'll use the sigmoid function from the Scipy package when making predictions.

In [20]:
from scipy.special import expit

def score_row(row):
    prob = expit(row.sum())
    if prob <= 0.5:
        return 0
    else:
        return 1
    
query_preds = test.apply(score_row, axis = 1)

query_preds

0         0
1         0
2         0
3         0
4         0
         ..
100650    1
100651    1
100652    1
100653    1
100654    1
Length: 100655, dtype: int64

Now, to test whether these predictions are right, I'll run predictions from the Xgboost model I just trained...

In [21]:
preds = model.predict(df.iloc[:, 0:41])
preds

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

Let's see, how often are these predictions the same?

In [22]:
(query_preds == preds).sum()

100655

And how many rows are in the dataset?

In [23]:
test.shape[0]

100655

BOOM! 

This passed the test.

Now, I'll close the connection the database.

In [24]:
conn.close()

Well, there you have it. If you have an Xgboost model, you can run model predictions from the database. No need to make a web endpoint to make predictions.